In [1]:
import re
import json
from ast import literal_eval
from collections import Counter
import pandas as pd
from tqdm.auto import tqdm

In [2]:
exportdata = pd.read_csv("exportdata_of_entries_of_projects_of_interest.csv")
entries = pd.read_csv("entries_of_projects_of_interest.csv")
af_widgets = pd.read_csv("widgets_of_afs_of_interest.csv")
projects = pd.read_csv("projects_of_interest.csv")
exportables = pd.read_csv("afexportable_of_af_of_projects_of_interest.csv")
##
afs = pd.read_csv("all_afs.csv")

/home/abdullah/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
entries.shape, exportdata.shape

((190907, 20), (1002149, 4))

In [4]:
entries[~entries["excerpt"].isna()]["excerpt"].unique().shape

(133348,)

In [5]:
exid_to_exdata = dict()
for ex_id, ex_data in zip(exportables["id"], exportables["data"]):
    exid_to_exdata[ex_id] = json.loads(ex_data)

In [6]:
af_widgets["title"].unique()

array(['SPECIFIC NEEDS GROUPS', 'DEMOGRAPHIC GROUPS', 'SEVERITY',
       'AFFECTED GROUPS', 'RELIABILITY', 'DATE OF INFORMATION', 'EXCERPT',
       'Cross Sectors', 'LOCATION', 'Sectors', 'Demographic Groups',
       'Cross sector', 'Severity', 'Affected groups', 'Geo Location',
       'Specific Needs Groups', 'Excerpt', 'Information date',
       'Reliability', 'Matrix 1D', 'Matrix 2D', 'Flag',
       'Operational Environment', 'Information Date',
       'Sectoral Information', 'Crisis Type', 'Context additional tags',
       'Crisis type', 'HIGH LEVEL TAGS', 'Cleaning comments',
       'Additional Context', 'POPULATION GROUPS', 'CLEANING tags',
       'DISPLACED POP TYPE', 'SHOCK/EVENT', 'PRE-CRISIS', 'IN-CRISIS',
       'Comment'], dtype=object)

In [7]:
widget_key_id_to_title = dict()
for w_key, w_id, title in zip(af_widgets["key"], af_widgets["widget_id"], af_widgets["title"]):
    widget_key_id_to_title[(w_key, w_id)] = title

In [8]:
def exportdata_to_tag_title(row):
    data = json.loads(row[1])
    wkey, wid = None, None
    if data.get("common"):
        wkey = data.get("common").get("widget_key")
        wid = data["common"].get("widget_id")
    if wkey and wid:
        if widget_key_id_to_title.get((wkey, wid)):
            return widget_key_id_to_title.get((wkey, wid))
    if data.get("report") and data["report"].get("other"):
            if len(data["report"]["other"]) == 1 and data["report"]["other"][0].get("title"):
                if data["report"]["other"][0]["title"]:
                    return data["report"]["other"][0]["title"]
    if isinstance(data.get("excel"), list) and len(data["excel"])==1 and \
     data["excel"][0].get("widget_key") and data["excel"][0].get("widget_id"):
        wkey = data["excel"][0]["widget_key"]
        wid = data["excel"][0]["widget_id"]
        if widget_key_id_to_title.get((wkey, wid)):
            return widget_key_id_to_title.get((wkey, wid))
    exportable_id = row[3]
    if exid_to_exdata[exportable_id]["excel"].get("title"):
        return exid_to_exdata[exportable_id]["excel"]["title"]
    elif exid_to_exdata[exportable_id]["excel"].get("type") == "multiple":
        return exid_to_exdata[exportable_id]["excel"]["titles"]
    raise

In [9]:
exportdata["tag_title"] = exportdata.apply(exportdata_to_tag_title, axis=1)

In [10]:
def exportdata_to_tag_value(row):
    data = json.loads(row[1])
    if isinstance(data["excel"], list) and len(data["excel"])==1:
        return data["excel"][0]["value"]
    if data["excel"].get("type") == "lists":
        return data["excel"]["values"]
    if isinstance(data["excel"], dict) and "value" in data["excel"]:
        return data["excel"].get("value")
    if isinstance(data["excel"], dict) and "values" in data["excel"]:
        return data["excel"].get("values")
    elif "values" in data["common"]:
        return data["common"]["values"]
    elif "value" in data["common"]:
        return data["common"]["value"]
    raise

In [11]:
exportdata["tag_value"] = exportdata.apply(exportdata_to_tag_value, axis=1)

In [12]:
def title_case(tag):
    if isinstance(tag, (list, tuple)):
        return tuple([x.title() for x in tag])
    return tag.title()

In [13]:
exportdata["tag_title"] = exportdata["tag_title"].apply(title_case)

In [14]:
exportdata["tag_title"].unique()
# 1. GIMAC --> Pre-Crisis, Shock/Event, In-Crisis
# 2. 2020 Okular --> Sectoral Information
# 3. Okular Analytics Generic --> Sectoral Information
# 4. Rohingya Framework --> Sectors
# 5. IFRC 2018 --> Sectors
# 6. Colombia AF --> Sectoral Information
# 7. Nigeria Situation Analysis (OA) --> Sectoral Information
# 8. Situation Analysis Generic Yemen --> Matrix 2D
# 9. Situation Analysis Generic Libya --> Sectors

array(['Sectoral Information', 'Affected Groups', 'Location',
       'Reliability', 'Demographic Groups', 'Information Date',
       'Severity', 'Flag', 'Operational Environment', 'Shock/Event',
       'High Level Tags', 'In-Crisis', 'Specific Needs Groups',
       'Additional Context', 'Sectors', 'Cross Sector',
       'Date Of Information', 'Geo Location', 'Displaced Pop Type',
       'Crisis Type', 'Displacement Profile Details', 'Date',
       'Cleaning Tags',
       ('Affected Groups - Level 0', 'Affected Groups - Level 1', 'Affected Groups - Level 2', 'Affected Groups - Level 3'),
       'Comment', 'Cleaning Comments',
       ('Sectoral Information - Dimension', 'Sectoral Information - Subdimension', 'Sectoral Information - Sector', 'Sectoral Information - Subsectors'),
       ('Information Date (From)', 'Information Date (To)'),
       ('Matrix 1D - Dimension', 'Matrix 1D - Subdimension'),
       ('Operational Environment - Dimension', 'Operational Environment - Subdimension'),


In [15]:
#exportdata[exportdata["tag_title"] == ('Matrix 2D - Dimension', 'Matrix 2D - Subdimension', 'Matrix 2D - Sector', 'Matrix 2D - Subsectors')]["tag_value"].tolist()

In [16]:
mat2d = [
    "Sectoral Information",
    "Shock/Event",
    "In-Crisis",
    "Sectors",
    ('Sectoral Information - Dimension', 'Sectoral Information - Subdimension',
     'Sectoral Information - Sector', 'Sectoral Information - Subsectors'),
    #('Matrix 2D - Dimension', 'Matrix 2D - Subdimension', 'Matrix 2D - Sector', 'Matrix 2D - Subsectors'),#only 3 labels one of them is noisy
    ('Dimension', 'Subdimension', 'Sector', 'Subsectors'),
    ('In-Crisis - Dimension', 'In-Crisis - Subdimension', 'In-Crisis - Sector', 'In-Crisis - Subsectors'),
    ('Sectors - Dimension', 'Sectors - Subdimension', 'Sectors - Sector', 'Sectors - Subsectors'),
    #('2D Matrix - Dimension', '2D Matrix - Subdimension', '2D Matrix - Sector', '2D Matrix - Subsectors'),#very different dimensions and sectors
    #('Pre-Crisis - Dimension', 'Pre-Crisis - Subdimension', 'Pre-Crisis - Sector', 'Pre-Crisis - Subsectors'),#empty
]

In [17]:
exportdata_mat2d = exportdata[exportdata["tag_title"].isin(mat2d)]

In [18]:
exportdata_mat2d.shape, exportdata.shape, entries.shape

((151165, 6), (1002149, 6), (190907, 20))

In [19]:
entries.columns

Index(['id', 'created_at', 'modified_at', 'excerpt', 'image_raw',
       'analysis_framework_id', 'created_by_id', 'lead_id', 'modified_by_id',
       'entry_type', 'information_date', 'order', 'client_id', 'project_id',
       'tabular_field_id', 'dropped_excerpt', 'highlight_hidden', 'verified',
       'verification_last_changed_by_id', 'image_id'],
      dtype='object')

In [20]:
exportdata.columns

Index(['id', 'data', 'entry_id', 'exportable_id', 'tag_title', 'tag_value'], dtype='object')

In [21]:
entries_labeled_mat2d = pd.merge(entries,
                                 exportdata_mat2d,
                                 how="inner",
                                 left_on="id",
                                 right_on="entry_id",
                                 suffixes=('_entry', '_exportdata'))

In [22]:
entries["id"].unique().shape, entries.shape

((190907,), (190907, 20))

In [23]:
entries_labeled_mat2d.shape, entries_labeled_mat2d.columns

((151135, 26),
 Index(['id_entry', 'created_at', 'modified_at', 'excerpt', 'image_raw',
        'analysis_framework_id', 'created_by_id', 'lead_id', 'modified_by_id',
        'entry_type', 'information_date', 'order', 'client_id', 'project_id',
        'tabular_field_id', 'dropped_excerpt', 'highlight_hidden', 'verified',
        'verification_last_changed_by_id', 'image_id', 'id_exportdata', 'data',
        'entry_id', 'exportable_id', 'tag_title', 'tag_value'],
       dtype='object'))

In [24]:
def mat2d_labels_to_dimension(tag_value):
    if not isinstance(tag_value, list):
        print(tag_value)
        raise
    if len(tag_value):
        dimensions = []
        for t in tag_value:
            if t[0]:
                dimensions.append(t[0].title())
        return list(set(dimensions))
    else:
        return []

def mat2d_labels_to_subdimension(tag_value):
    if not isinstance(tag_value, list):
        print(tag_value)
        raise
    if len(tag_value):
        subdimensions = []
        for t in tag_value:
            if t[1]:
                subdimensions.append(t[1].title())
        return list(set(subdimensions))
    else:
        return []

def mat2d_labels_to_sector(tag_value):
    if not isinstance(tag_value, list):
        print(tag_value)
        raise
    if len(tag_value):
        sectors = []
        for t in tag_value:
            if t[2]:
                sectors.append(t[2].title())
        return list(set(sectors))
    else:
        return []

def mat2d_labels_to_subsector(tag_value):
    if not isinstance(tag_value, list):
        print(tag_value)
        raise
    if len(tag_value):
        subsectors = []
        for t in tag_value:
            if len(t) == 4 and t[3]:
                if isinstance(t[3], list):
                    subsectors.extend([x.title() for x in t[3]])
                else:
                    subsectors.append(t[3].title())
        return list(set(subsectors))
    else:
        return []

In [25]:
entries_labeled_mat2d["Dimension"] = entries_labeled_mat2d["tag_value"].apply(mat2d_labels_to_dimension)
entries_labeled_mat2d["Subdimension"] = entries_labeled_mat2d["tag_value"].apply(mat2d_labels_to_subdimension)
entries_labeled_mat2d["Sector"] = entries_labeled_mat2d["tag_value"].apply(mat2d_labels_to_sector)
entries_labeled_mat2d["Subsector"] = entries_labeled_mat2d["tag_value"].apply(mat2d_labels_to_subsector)

In [26]:
dims_unique = set()
dims_occurances = list()
for dims in entries_labeled_mat2d["Dimension"]:
    dims_unique.update(dims)
    dims_occurances.extend(dims)
#dims_unique
Counter(dims_occurances).most_common()

[('Humanitarian Conditions', 71862),
 ('Impact', 28375),
 ('Operational Environment', 10460),
 ('Capacities & Response', 10119),
 ('Effects On Population', 7783),
 ('Response And Capacities', 7150),
 ('At Risk', 5625),
 ('Effects Systems And Networks', 5110),
 ('Scope & Scale', 4951),
 ('Priorities', 4895),
 ('Humanitarian Situation', 4646),
 ('Capacities And Response', 4338),
 ('Context', 4330),
 ('Stated Priorities And Recommendations', 4313),
 ('Shock Informaton', 3306),
 ('Scope And Scale', 2688),
 ('Coping Capacity', 1591)]

In [27]:
secs_unique = set()
secs_occurances = list()
for secs in entries_labeled_mat2d["Sector"]:
    secs_unique.update(secs)
    secs_occurances.extend(secs)
#secs_unique
Counter(secs_occurances).most_common()

[('Health', 43868),
 ('Protection', 31370),
 ('Cross-Sector', 19260),
 ('Livelihoods', 18781),
 ('Food Security', 15635),
 ('Wash', 14440),
 ('Cross', 14314),
 ('Education', 11859),
 ('Shelter', 10080),
 ('Nutrition', 6725),
 ('Food', 4514),
 ('Logistics', 3059),
 ('Agriculture', 2795),
 ('Shelter And Nfis', 2134),
 ('Nfi', 1330),
 ('Cross Cutting', 869),
 ('Cccm', 731),
 ('Livelihood', 576),
 ('Logistic', 306)]

In [28]:
entries_labeled_mat2d["Subdimension"]

0         [Number Of People Affected, Impact On System &...
1                                           [International]
2                                           [International]
3              [Impact On System & Services, International]
4                 [Number Of People Reached, International]
                                ...                        
151130                             [International Response]
151131    [Specific Needs, Access To Basic Services Or G...
151132                             [International Response]
151133                             [International Response]
151134    [International Response, National Response, Ri...
Name: Subdimension, Length: 151135, dtype: object

In [29]:
entries_labeled_mat2d["Sector"].apply(lambda x: True if x else False).sum()

147044

In [30]:
entries_labeled_mat2d["Sector"].apply(lambda x: True if len(x)>1 else False).sum()

35116

In [31]:
entries_labeled_mat2d.columns

Index(['id_entry', 'created_at', 'modified_at', 'excerpt', 'image_raw',
       'analysis_framework_id', 'created_by_id', 'lead_id', 'modified_by_id',
       'entry_type', 'information_date', 'order', 'client_id', 'project_id',
       'tabular_field_id', 'dropped_excerpt', 'highlight_hidden', 'verified',
       'verification_last_changed_by_id', 'image_id', 'id_exportdata', 'data',
       'entry_id', 'exportable_id', 'tag_title', 'tag_value', 'Dimension',
       'Subdimension', 'Sector', 'Subsector'],
      dtype='object')

In [32]:
entries_labeled_mat2d["entry_type"].unique()

array(['image', 'excerpt', 'dataSeries'], dtype=object)

In [33]:
entries_labeled_mat2d[entries_labeled_mat2d["entry_type"].eq("excerpt")]["excerpt"].isna().sum()

42

In [34]:
final_df = entries_labeled_mat2d[
    entries_labeled_mat2d["entry_type"].eq("excerpt")
    & (~entries_labeled_mat2d["excerpt"].isna())][[
        'id_entry', 'excerpt', 'analysis_framework_id', 'created_by_id',
        'lead_id', 'modified_by_id', 'information_date', 'order', 'client_id',
        'project_id', 'tabular_field_id', 'dropped_excerpt', 'verified',
        'verification_last_changed_by_id', 'id_exportdata', 'exportable_id',
        'Dimension', 'Subdimension', 'Sector', 'Subsector'
    ]]

In [35]:
final_df.columns = [
    'entry_id', 'excerpt', 'analysis_framework_id', 'created_by_id', 'lead_id',
    'modified_by_id', 'information_date', 'order', 'client_id', 'project_id',
    'tabular_field_id', 'dropped_excerpt', 'verified',
    'verification_last_changed_by_id', 'exportdata_id', 'exportable_id',
    'dimensions', 'subdimensions', 'sectors', 'subsectors'
]

In [36]:
def remove_newlines(excerpt):
    if not isinstance(excerpt, str):
        return excerpt
    return re.sub("\s+", " ", excerpt)

final_df['excerpt'] = final_df['excerpt'].apply(remove_newlines)
final_df['dropped_excerpt'] = final_df['dropped_excerpt'].apply(remove_newlines)
##
final_df = final_df.sort_values("verified").reset_index(drop=True).drop_duplicates(subset='excerpt', keep="last")
final_df.shape, final_df.duplicated(subset="excerpt").sum()

((107568, 20), 0)

### Data Cleaning

In [37]:
dimension_name_mapper = {
    'At Risk': "At Risk",
    'Capacities & Response': 'Capacities & Response',
    'Capacities And Response': 'Capacities & Response',
    'Response And Capacities': 'Capacities & Response',
    'Coping Capacity': 'Capacities & Response',
    'Context': "Context",
    'Operational Environment': 'Context',
    'Effects On Population': 'Effects On Population',
    'Effects Systems And Networks': 'Effects Systems And Networks',
    'Humanitarian Conditions': 'Humanitarian Conditions',
    'Humanitarian Situation': 'Humanitarian Conditions',
    'Impact': 'Impact',
    'Priorities': 'Priorities',
    'Stated Priorities And Recommendations': 'Priorities',
    'Scope & Scale': 'Scope & Scale',
    'Scope And Scale': 'Scope & Scale',
    'Shock Informaton': 'Shock Informaton',
}

In [38]:
sector_name_mapper = {
    "Agriculture": "Agriculture",
    "Cccm": "CCCM",
    "Cross": "Cross",
    "Cross Cutting": "Cross",
    "Cross-Sector": "Cross",
    "Education": "Education",
    "Food": "Food Security",
    "Food Security": "Food Security",
    "Nutrition": "Nutrition",
    "Health": "Health",
    "Livelihood": "Livelihoods",
    "Livelihoods": "Livelihoods",
    "Logistic": "Logistics",
    "Logistics": "Logistics",
    "Protection": "Protection",
    "Shelter": "Shelter",
    "Shelter And Nfis": "Shelter and NFIs",
    "Nfi": "NFI",
    "Wash": "WASH",
}

In [39]:
def sector_mapper(sec):
    if sec == sec:
        return [sector_name_mapper.get(s, s) for s in sec]
    return []
##
def dimension_mapper(dim):
    if dim == dim:
        return [dimension_name_mapper.get(d, d) for d in dim]
    return []

In [40]:
final_df["sectors"] = final_df["sectors"].apply(sector_mapper)
final_df["dimensions"] = final_df["dimensions"].apply(dimension_mapper)

In [41]:
sectors = set()
for sec in final_df["sectors"]:
    if sec and sec==sec:
        sectors.update(sec)
sectors

{'Agriculture',
 'CCCM',
 'Cross',
 'Education',
 'Food Security',
 'Health',
 'Livelihoods',
 'Logistics',
 'NFI',
 'Nutrition',
 'Protection',
 'Shelter',
 'Shelter and NFIs',
 'WASH'}

In [42]:
dimensions = set()
for dim in final_df["dimensions"]:
    if dim and dim==dim:
        dimensions.update(dim)
dimensions

{'At Risk',
 'Capacities & Response',
 'Context',
 'Effects On Population',
 'Effects Systems And Networks',
 'Humanitarian Conditions',
 'Impact',
 'Priorities',
 'Scope & Scale',
 'Shock Informaton'}

In [43]:
sec_to_id = {sec:i for i, sec in enumerate(sectors)}
sec_to_id

{'Cross': 0,
 'Food Security': 1,
 'Agriculture': 2,
 'Livelihoods': 3,
 'Shelter and NFIs': 4,
 'Shelter': 5,
 'Nutrition': 6,
 'WASH': 7,
 'Health': 8,
 'Education': 9,
 'Logistics': 10,
 'Protection': 11,
 'NFI': 12,
 'CCCM': 13}

In [44]:
dim_to_id = {dim:i for i, dim in enumerate(dimensions)}
dim_to_id

{'At Risk': 0,
 'Impact': 1,
 'Effects On Population': 2,
 'Scope & Scale': 3,
 'Humanitarian Conditions': 4,
 'Priorities': 5,
 'Context': 6,
 'Effects Systems And Networks': 7,
 'Shock Informaton': 8,
 'Capacities & Response': 9}

In [45]:
final_df['Shelter'] = 0
final_df['Livelihoods'] = 0
final_df['Protection'] = 0
final_df['Shelter and NFIs'] = 0
final_df['NFI'] = 0
final_df['Food Security'] = 0
final_df['Logistics'] = 0
final_df['Cross'] = 0
final_df['CCCM'] = 0
final_df['Agriculture'] = 0
final_df['Health'] = 0
final_df['Nutrition'] = 0
final_df['Education'] = 0
final_df['WASH'] = 0
final_df['sector_ids'] = ""

In [46]:
final_df['Shock Informaton'] = 0
final_df['Effects Systems And Networks'] = 0
final_df['Effects On Population'] = 0
final_df['Capacities & Response'] = 0
final_df['At Risk'] = 0
final_df['Scope & Scale'] = 0
final_df['Impact'] = 0
final_df['Humanitarian Conditions'] = 0
final_df['Priorities'] = 0
final_df['Context'] = 0
final_df["dimension_ids"] = ""

In [47]:
for row in tqdm(final_df.iterrows(), total=final_df.shape[0]):
    sec = row[1]["sectors"]
    idx = row[0]
    sec_ids = []
    if 'Agriculture' in sec:
        final_df.loc[idx, 'Agriculture'] = 1
        sec_ids.append(sec_to_id["Agriculture"])
    if 'CCCM' in sec:
        final_df.loc[idx, 'CCCM'] = 1
        sec_ids.append(sec_to_id["CCCM"])
    if 'Cross' in sec:
        final_df.loc[idx, 'Cross'] = 1
        sec_ids.append(sec_to_id["Cross"])
    if 'Education' in sec:
        final_df.loc[idx, 'Education'] = 1
        sec_ids.append(sec_to_id["Education"])
    if 'Food Security' in sec:
        final_df.loc[idx, 'Food Security'] = 1
        sec_ids.append(sec_to_id["Food Security"])
    if 'Nutrition' in sec:
        final_df.loc[idx, 'Nutrition'] = 1
        sec_ids.append(sec_to_id["Nutrition"])
    if 'Health' in sec:
        final_df.loc[idx, 'Health'] = 1
        sec_ids.append(sec_to_id["Health"])
    if 'Livelihoods' in sec:
        final_df.loc[idx, 'Livelihoods'] = 1
        sec_ids.append(sec_to_id["Livelihoods"])
    if 'Logistics' in sec:
        final_df.loc[idx, 'Logistics'] = 1
        sec_ids.append(sec_to_id["Logistics"])
    if 'Protection' in sec:
        final_df.loc[idx, 'Protection'] = 1
        sec_ids.append(sec_to_id["Protection"])
    if 'Shelter' in sec:
        final_df.loc[idx, 'Shelter'] = 1
        sec_ids.append(sec_to_id["Shelter"])
    if 'Shelter and NFIs' in sec:
        final_df.loc[idx, 'Shelter and NFIs'] = 1
        sec_ids.append(sec_to_id["Shelter and NFIs"])
    if 'NFI' in sec:
        final_df.loc[idx, 'NFI'] = 1
        sec_ids.append(sec_to_id["NFI"])
    if 'WASH' in sec:
        final_df.loc[idx, 'WASH'] = 1
        sec_ids.append(sec_to_id["WASH"])
    final_df.loc[idx, 'sector_ids'] = str(sec_ids)

100%|██████████| 107568/107568 [04:23<00:00, 408.69it/s]


In [48]:
for row in tqdm(final_df.iterrows(), total=final_df.shape[0]):
    dim = row[1]["dimensions"]
    idx = row[0]
    dim_ids = []
    if "Shock Informaton" in dim:
        final_df.loc[idx, 'Shock Informaton'] = 1
        dim_ids.append(dim_to_id["Shock Informaton"])
    if "Effects Systems And Networks" in dim:
        final_df.loc[idx, 'Effects Systems And Networks'] = 1
        dim_ids.append(dim_to_id["Effects Systems And Networks"])
    if "Effects On Population" in dim:
        final_df.loc[idx, 'Effects On Population'] = 1
        dim_ids.append(dim_to_id["Effects On Population"])
    if "Capacities & Response" in dim:
        final_df.loc[idx, 'Capacities & Response'] = 1
        dim_ids.append(dim_to_id["Capacities & Response"])
    if "At Risk" in dim:
        final_df.loc[idx, 'At Risk'] = 1
        dim_ids.append(dim_to_id["At Risk"])
    if "Scope & Scale" in dim:
        final_df.loc[idx, 'Scope & Scale'] = 1
        dim_ids.append(dim_to_id["Scope & Scale"])
    if "Impact" in dim:
        final_df.loc[idx, 'Impact'] = 1
        dim_ids.append(dim_to_id["Impact"])
    if "Humanitarian Conditions" in dim:
        final_df.loc[idx, 'Humanitarian Conditions'] = 1
        dim_ids.append(dim_to_id["Humanitarian Conditions"])
    if "Priorities" in dim:
        final_df.loc[idx, 'Priorities'] = 1
        dim_ids.append(dim_to_id["Priorities"])
    if "Context" in dim:
        final_df.loc[idx, 'Context'] = 1
        dim_ids.append(dim_to_id["Context"])
    final_df.loc[idx, 'dimension_ids'] = str(dim_ids)

100%|██████████| 107568/107568 [03:29<00:00, 514.35it/s]


In [49]:
final_df['sector_ids'] = final_df['sector_ids'].apply(literal_eval)
final_df['dimension_ids'] = final_df['dimension_ids'].apply(literal_eval)

### Stratified Split

In [50]:
import random
from collections import Counter
import numpy as np
import pandas as pd
from skmultilearn.model_selection import iterative_train_test_split
from skmultilearn.model_selection.measures import get_combination_wise_output_matrix

In [51]:
final_df[[
    'Shelter',
    'Livelihoods',
    'Protection',
    'Shelter and NFIs',
    'NFI',
    'Food Security',
    'Logistics',
    'Cross',
    'CCCM',
    'Agriculture',
    'Health',
    'Nutrition',
    'Education',
    'WASH',
]].sum().astype(np.int)

Shelter              6398
Livelihoods         13390
Protection          23753
Shelter and NFIs     1821
NFI                   725
Food Security       13270
Logistics            2277
Cross               25045
CCCM                  604
Agriculture          2225
Health              30611
Nutrition            4694
Education            8397
WASH                10048
dtype: int64

In [52]:
final_df.shape

(107568, 46)

In [53]:
def train_val_test_df_split(df):
    num_sectors = len(sec_to_id)
    num_classes = num_sectors + len(dim_to_id)
    labels = np.zeros([len(df), num_classes])
    for i, (sec_ids, dim_ids) in enumerate(zip(df["sector_ids"], df["dimension_ids"])):
        for sec_id in sec_ids:
            labels[i, sec_id] = 1
        for dim_id in dim_ids:
            labels[i, dim_id] = 1
    X_train, y_train, X_test, y_test = iterative_train_test_split(
        df["excerpt"].to_numpy().reshape(-1, 1), labels, test_size=0.1)
    X_train, y_train, X_val, y_val = iterative_train_test_split(X_train,
                                                                y_train,
                                                                test_size=0.1)
    return {
        "X_train": X_train,
        "y_train": y_train,
        "X_test": X_test,
        "y_test": y_test,
        "X_val": X_val,
        "y_val": y_val
    }

In [54]:
X_train, y_train, X_test, y_test, X_val, y_val = list(
    train_val_test_df_split(final_df).values())

In [55]:
final_df_train = final_df[final_df["excerpt"].isin(X_train.reshape(-1))]
final_df_val = final_df[final_df["excerpt"].isin(X_val.reshape(-1))]
final_df_test = final_df[final_df["excerpt"].isin(X_test.reshape(-1))]

In [56]:
final_df_train.index & final_df_val.index, final_df_train.index & final_df_test.index, final_df_test.index & final_df_val.index 

(Int64Index([], dtype='int64'),
 Int64Index([], dtype='int64'),
 Int64Index([], dtype='int64'))

In [57]:
final_df_train.to_csv("data_v0.3_train.csv", index=None)
final_df_val.to_csv("data_v0.3_val.csv", index=None)
final_df_test.to_csv("data_v0.3_test.csv", index=None)